<a href="https://colab.research.google.com/github/apujais007/api-nuggets/blob/main/M%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
import os

# Set the OPENAI_API_KEY environment variable
os.environ["FMP_API_KEY"] = userdata.get('FMP')

print("FMP_API_KEY environment variable has been set.")

FMP_API_KEY environment variable has been set.


In [14]:
import requests

# Define the API endpoint and your API key
api_url = "https://financialmodelingprep.com/stable/actively-trading-list"
api_key = os.environ["FMP_API_KEY"]

# Send a GET request to the API
response = requests.get(api_url, params={"apikey": api_key})

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response to include only symbol names
    data = response.json()

    # Filter for NYSE-listed stocks
    nyse_stocks = [stock['symbol'] for stock in data]
else:
    print(f"Error fetching data: {response.status_code}")

#print(len(nyse_stocks))
#print(nyse_stocks)


In [16]:
import pandas as pd
from datetime import datetime, date

def fetch_today_ma(symbols, api_key, debug=False, test_date=None):
    """
    Fetch mergers & acquisitions for given symbols and return rows where
    transactionDate or acceptedDate matches today (or test_date if provided).

    Parameters:
        symbols (list): List of symbols to check
        api_key (str): FMP API key
        debug (bool): If True, prints debug info
        test_date (datetime.date): Optional date to override 'today' for testing

    Returns:
        pd.DataFrame: DataFrame with MA info for matched symbols
    """
    if test_date:
        today = test_date
    else:
        today = date.today()

    # API URL
    url = f"https://financialmodelingprep.com/stable/mergers-acquisitions-latest?page=0&limit=100&apikey={api_key}"

    # Fetch data
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Error fetching API: {response.status_code}")

    ma_data = response.json()

    if debug:
        print(f"Total records fetched: {len(ma_data)}")

    results = []
    for record in ma_data:
        symbol = record.get("symbol")
        if symbol not in symbols:
            continue

        # Parse dates
        trans_date_str = record.get("transactionDate")
        acc_date_str = record.get("acceptedDate")
        try:
            trans_date = datetime.strptime(trans_date_str[:10], "%Y-%m-%d").date() if trans_date_str else None
            acc_date = datetime.strptime(acc_date_str[:10], "%Y-%m-%d").date() if acc_date_str else None
        except Exception as e:
            if debug:
                print(f"Error parsing dates for {symbol}: {e}")
            continue

        # Check if today matches
        if today in [trans_date, acc_date]:
            results.append(record)
            if debug:
                print(f"Matched symbol: {symbol}, transactionDate: {trans_date}, acceptedDate: {acc_date}")

    if results:
        df = pd.DataFrame(results)
    else:
        df = pd.DataFrame(columns=ma_data[0].keys() if ma_data else [])

    return df

# -----------------------------
# Example usage
# -----------------------------
# symbols_to_check = ["CBAN", "UNP", "NSC"]  # replace with your symbols list
API_KEY = os.environ["FMP_API_KEY"]
#df_today_ma = fetch_today_ma(nyse_stocks, API_KEY, debug=True, test_date=date(2025, 9, 5))
df_today_ma = fetch_today_ma(nyse_stocks, API_KEY, debug=True).drop(columns=["cik","targetedCik"],errors="ignore")
display(df_today_ma)


Total records fetched: 100


,symbol,companyName,targetedCompanyName,targetedSymbol,transactionDate,acceptedDate,link
